# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [4]:
#!pip install pandas

In [5]:
import pandas as pd

# Specify the URL of the CSV file
url = 'https://github.com/ironhack-labs/lab-chains-in-langchain/raw/main/data/Data.csv'

# Read CSV with specific parameters
try:
    df = pd.read_csv(url, delimiter=',')  # Adjust delimiter if necessary
    print("CSV file loaded successfully.")
    print(df.head())  # Print first few rows to verify
except Exception as e:
    print(f"Error loading CSV file: {e}")


CSV file loaded successfully.
                   Product                                             Review
0     Queen Size Sheet Set  I ordered a king size set. My only criticism w...
1   Waterproof Phone Pouch  I loved the waterproof sac, although the openi...
2      Luxury Air Mattress  This mattress had a small hole in the top of i...
3           Pillows Insert  This is the best throw pillow fillers on Amazo...
4  Milk Frother Handheld\n   I loved this product. But they only seem to l...


In [6]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [8]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [9]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0.5)

# Justification:
# A temperature of 0.5 balances between generating diverse responses
# and maintaining coherence. It introduces a moderate level of randomness,
# allowing for varied responses without straying too far from relevant content.

In [10]:
# Define a variable placeholder for product description
product_description = "{product}"

# Construct the prompt template
prompt = ChatPromptTemplate.from_template(
    f"Write a query that would take a variable to describe any product.\n\n"
    f"Describe the product: {product_description}\n\n"
    f"Example: Describe the features, specifications, and benefits of {product_description}."
)
print("Prompt template constructed successfully.")


Prompt template constructed successfully.


In [11]:
chain = LLMChain(llm=llm, prompt=prompt)

/opt/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [12]:
product = "Queen Size Sheet Set"
#chain.run(product)
llm.invoke(product)


AIMessage(content='A queen size sheet set typically includes a fitted sheet, flat sheet, and two standard pillowcases. These sheet sets are designed to fit a standard queen size mattress, which measures 60 inches by 80 inches. Queen size sheet sets come in a variety of materials, such as cotton, microfiber, or bamboo, and can feature different thread counts for varying levels of softness and durability. They are available in a wide range of colors and patterns to suit any bedroom decor.', response_metadata={'token_usage': {'completion_tokens': 96, 'prompt_tokens': 11, 'total_tokens': 107}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-87b8fbad-44d8-45b4-a4cf-c918426c5957-0', usage_metadata={'input_tokens': 11, 'output_tokens': 96, 'total_tokens': 107})

In [13]:
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain_core.prompts import PromptTemplate

In [14]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)
# Chain 1
chain_one = LLMChain(llm=llm, prompt=prompt)

In [15]:
# prompt template 2
second_prompt = PromptTemplate(
    input_variables=["company_name"],
    template="Write a description of a logo for this company: {company_name}, the logo should not contain text at all ",
)
# Chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [16]:
# Create the SimpleSequentialChain
overall_simple_chain = SimpleSequentialChain(
    chains=[chain_one, chain_two], verbose=True
)

In [17]:
overall_simple_chain.run(product)

/opt/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new SimpleSequentialChain chain...
Regal Rest Bedding Co.
The logo for Regal Rest Bedding Co. features a regal and luxurious crown design. The intricate crown symbolizes the superior quality and comfort of the bedding products offered by the company. The use of gold and purple colors adds a touch of elegance and sophistication to the logo, reflecting the premium nature of the brand. The overall design is sleek and timeless, making it instantly recognizable and memorable.

> Finished chain.


'The logo for Regal Rest Bedding Co. features a regal and luxurious crown design. The intricate crown symbolizes the superior quality and comfort of the bedding products offered by the company. The use of gold and purple colors adds a touch of elegance and sophistication to the logo, reflecting the premium nature of the brand. The overall design is sleek and timeless, making it instantly recognizable and memorable.'

**Repeat the above twice for different products**

## SequentialChain

In [20]:
from langchain.chains import SequentialChain

In [21]:
llm = ChatOpenAI(temperature=0.9)


# Define prompt template 1: Translate a review into English
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review into English:\n\n{review_text}\n\n"
)

# Chain 1: Translate review
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="translated_review")


In [22]:
# Define prompt template 2: Summarize a review
second_prompt = ChatPromptTemplate.from_template(
    "Write a summary of the translated review: {translated_review}"
)

# Chain 2: Summarize translated review
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="summary")


In [23]:
# prompt template 3: translate to english or other language
# Define prompt template 3: Translate to another language (example prompt)
third_prompt = ChatPromptTemplate.from_template(
    "Translate the summary into another language: {summary}"
)

# Chain 3: Translate summary to another language
chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key="translated_summary")


In [24]:
# prompt template 4: follow up message that take as inputs the two previous prompts' variables
# Define prompt template 4: Generate a follow-up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Given the translated review:\n\n{translated_review}\n\n"
    "and its summary:\n\n{summary}\n\n"
    "please generate a follow-up message responding to the review."
)

# Chain 4: Generate follow-up message
chain_four = LLMChain(llm=llm, prompt=fourth_prompt, output_key="follow_up_message")


In [25]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
# Overall SequentialChain: input= Review and output= translated_review, summary, translated_summary, follow_up_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review_text"],  # Input variable for the whole process
    output_variables=["translated_review", "summary", "translated_summary", "follow_up_message"],  # Output variables
    verbose=True
)


In [42]:
review = df.Review[5]
overall_chain.invoke(review)



> Entering new SequentialChain chain...

> Finished chain.


{'review_text': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'translated_review': "I find the taste mediocre. The foam doesn't hold, it's strange. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?",
 'summary': "The reviewer is disappointed with the taste of the product, describing it as mediocre. They also mention that the foam doesn't hold well, which they find strange. They compare the product to ones they buy in stores, which they believe have a much better taste. The reviewer questions whether the product they received may be from an old batch or could potentially be counterfeit.",
 'translated_summary': 'El revisor está decepcionado con el sabor del producto, describiéndolo como mediocre. También mencionan que la espuma no se mantiene bien, lo cual les resulta extraño. Comparan el producto con los que compran en tiendas,

**Repeat the above twice for different products or reviews**

## Router Chain

In [29]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [30]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

In [31]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [32]:
llm = ChatOpenAI(temperature=0)

In [33]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [34]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [35]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [36]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [37]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [38]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


"Black body radiation refers to the electromagnetic radiation emitted by a perfect black body, which is an idealized physical body that absorbs all incident electromagnetic radiation. The radiation emitted by a black body depends only on its temperature and follows a specific distribution known as Planck's law. This radiation is characterized by a continuous spectrum of wavelengths and intensities, with the peak intensity shifting to shorter wavelengths as the temperature of the black body increases. Black body radiation plays a key role in understanding concepts such as thermal radiation and the quantization of energy in quantum mechanics."

In [39]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'2 + 2 is equal to 4.'

In [40]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in our body contain DNA?'}

ValueError: Received invalid destination chain name 'biology'

**Repeat the above at least once for different inputs and chains executions - Be creative!**